In [1]:
%run ../ipynb_util_tars.py

In [2]:
%env WANDB_PROJECT=thesis

env: WANDB_PROJECT=thesis


In [3]:
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# LLaMA3 8B - unmasked

In [4]:
import torch.nn as nn
import torch
from transformers.models.llama.modeling_llama import LlamaForSequenceClassification, LlamaDecoderLayer, LlamaConfig, LlamaRMSNorm, LlamaPreTrainedModel, LlamaModel, LLAMA_INPUTS_DOCSTRING, add_start_docstrings_to_model_forward, SequenceClassifierOutputWithPast, BaseModelOutputWithPast, BCEWithLogitsLoss, CrossEntropyLoss, MSELoss
from transformers.cache_utils import Cache, DynamicCache
from typing import Optional, List, Union, Tuple


class UnmaskingLlamaModel(LlamaModel):
    """
    Transformer decoder consisting of *config.num_hidden_layers* layers. Each layer is a [`LlamaDecoderLayer`]

    Args:
        config: LlamaConfig
    """

    def __init__(self, config: LlamaConfig):
        super().__init__(config)
        self.padding_idx = config.pad_token_id
        self.vocab_size = config.vocab_size

        self.embed_tokens = nn.Embedding(config.vocab_size, config.hidden_size, self.padding_idx)
        self.layers = nn.ModuleList(
            [LlamaDecoderLayer(config, layer_idx) for layer_idx in range(config.num_hidden_layers)]
        )
        self.norm = LlamaRMSNorm(config.hidden_size, eps=config.rms_norm_eps)
        self.gradient_checkpointing = False

        # Initialize weights and apply final processing
        self.post_init()

    def get_input_embeddings(self):
        return self.embed_tokens

    def set_input_embeddings(self, value):
        self.embed_tokens = value

    @add_start_docstrings_to_model_forward(LLAMA_INPUTS_DOCSTRING)
    def forward(
        self,
        input_ids: torch.LongTensor = None,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        past_key_values: Optional[Union[Cache, List[torch.FloatTensor]]] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        cache_position: Optional[torch.LongTensor] = None,
    ) -> Union[Tuple, BaseModelOutputWithPast]:
        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )
        use_cache = use_cache if use_cache is not None else self.config.use_cache
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        if (input_ids is None) ^ (inputs_embeds is not None):
            raise ValueError(
                "You cannot specify both input_ids and inputs_embeds at the same time, and must specify either one"
            )

        if self.gradient_checkpointing and self.training and use_cache:
            logger.warning_once(
                "`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`."
            )
            use_cache = False

        if inputs_embeds is None:
            inputs_embeds = self.embed_tokens(input_ids)

        return_legacy_cache = False
        if use_cache and not isinstance(past_key_values, Cache):  # kept for BC (non `Cache` `past_key_values` inputs)
            return_legacy_cache = True
            past_key_values = DynamicCache.from_legacy_cache(past_key_values)

        if cache_position is None:
            past_seen_tokens = past_key_values.get_seq_length() if past_key_values is not None else 0
            cache_position = torch.arange(
                past_seen_tokens, past_seen_tokens + inputs_embeds.shape[1], device=inputs_embeds.device
            )
        if position_ids is None:
            position_ids = cache_position.unsqueeze(0)

        causal_mask = self._update_causal_mask(
            attention_mask, inputs_embeds, cache_position, past_key_values, output_attentions
        )
        if causal_mask is not None:
            #print("b4", input_ids.shape, causal_mask.shape, causal_mask)
            # Assuming causal_mask is a tensor with shape (batch_size, 1, seq_length, hidden_size)
            causal_mask_last_row = causal_mask[:, :, -1, :].unsqueeze(2)
            causal_mask = causal_mask_last_row.expand_as(causal_mask)
            # causal_mask = torch.zeros_like(causal_mask, device=inputs_embeds.device)

            #print("after", causal_mask.shape, causal_mask)
        else:
            pass
            #print("kek it's none", causal_mask, input_ids)

        # embed positions
        hidden_states = inputs_embeds

        # decoder layers
        all_hidden_states = () if output_hidden_states else None
        all_self_attns = () if output_attentions else None
        next_decoder_cache = None

        for decoder_layer in self.layers:
            if output_hidden_states:
                all_hidden_states += (hidden_states,)

            if self.gradient_checkpointing and self.training:
                layer_outputs = self._gradient_checkpointing_func(
                    decoder_layer.__call__,
                    hidden_states,
                    causal_mask,
                    position_ids,
                    past_key_values,
                    output_attentions,
                    use_cache,
                    cache_position,
                )
            else:
                layer_outputs = decoder_layer(
                    hidden_states,
                    attention_mask=causal_mask,
                    position_ids=position_ids,
                    past_key_value=past_key_values,
                    output_attentions=output_attentions,
                    use_cache=use_cache,
                    cache_position=cache_position,
                )

            hidden_states = layer_outputs[0]

            if use_cache:
                next_decoder_cache = layer_outputs[2 if output_attentions else 1]

            if output_attentions:
                all_self_attns += (layer_outputs[1],)

        hidden_states = self.norm(hidden_states)

        # add hidden states from the last decoder layer
        if output_hidden_states:
            all_hidden_states += (hidden_states,)

        next_cache = next_decoder_cache if use_cache else None
        if return_legacy_cache:
            next_cache = next_cache.to_legacy_cache()

        if not return_dict:
            return tuple(v for v in [hidden_states, next_cache, all_hidden_states, all_self_attns] if v is not None)
        return BaseModelOutputWithPast(
            last_hidden_state=hidden_states,
            past_key_values=next_cache,
            hidden_states=all_hidden_states,
            attentions=all_self_attns,
        )

class UnmaskingLlamaForSequenceClassification(LlamaForSequenceClassification):

    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.model = UnmaskingLlamaModel(config)
        self.score = nn.Linear(config.hidden_size, self.num_labels, bias=False)

        # Initialize weights and apply final processing
        self.post_init()

## Load ZO_UP data

In [5]:
# select dataset
from enum import Enum

class DatasetType(Enum):
    """Enum for the dataset type."""

    """Zora + OSDG upsampled dataset"""
    ZO_UP = "zo_up"
    """SwissText Shared Task 1 dataset (Zurich NLP)"""
    SWISSTEXT_SHARED_TASK1 = "swisstext_shared_task1"
    """Toy dataset for testing"""
    TOY = "toy_data"


DATASET_TYPE = DatasetType.ZO_UP

In [6]:
from datasets import load_dataset, Features, Value, ClassLabel, Dataset, DatasetDict
import pickle

# load the dataset
# note: if you don't have the data in the folder, use the download-data.sh script

match DATASET_TYPE:
    case DatasetType.TOY:
        dataset = DatasetDict({
            "train": Dataset.from_list([{"abstract": "hello world", "sdg": 1}, {"abstract": "Is this about clean energy?", "sdg": 7}]),
            "test": Dataset.from_list([{"abstract": "kek what is this", "sdg": 16}])
        })
        dataset = dataset.rename_columns({"sdg": "SDG", "abstract": "ABSTRACT"}).class_encode_column("SDG")

    case DatasetType.ZO_UP:
        # dont need to use manual features as class_encode_column will create ClassLabel
        # careful: watch out for the order of the ClassLabel as it doesn't map directly to the SDG class. need use mapping functions (id2label, label2id)
        # sdgs = [str(i) for i in range(1, 18)] + ["non-relevant"]
        # features = Features({"sdg": ClassLabel(num_classes=len(sdgs), names=sdgs), "abstract": Value("string")})

        dataset = load_dataset("csv", data_files=str(DATA_DIR_PATH / "zo_up.csv"))
        dataset = dataset.rename_columns({"sdg": "SDG", "abstract": "ABSTRACT"}).class_encode_column("SDG")
        dataset = dataset["train"].train_test_split(test_size=0.3, stratify_by_column="SDG", seed=SEED)
    case DatasetType.SWISSTEXT_SHARED_TASK1:
        dataset = load_dataset("json", data_files=str(DATA_DIR_PATH / "swisstext-2024-sharedtask" / "task1-train.jsonl"))
        dataset = dataset["train"].train_test_split(test_size=0.3, seed=SEED)

print(dataset["train"].features)
example = dataset["train"][0]
print("Example instance:\t", example)


# Label encodings / mappings
labels = set(dataset["train"]["SDG"])
id2label = {i: dataset["train"].features["SDG"].int2str(i) for i in range(len(labels))}
label2id = {dataset["train"].features["SDG"].int2str(i): i for i in range(len(labels))}

# save the encodings to a file for later use
ENCODING_DIR = BASE_DIR_PATH / "encodings" / DATASET_TYPE.value
# create the directory if it doesn't exist
ENCODING_DIR.mkdir(parents=True, exist_ok=True)
with open(ENCODING_DIR / "id2label.pkl", "wb") as f:
    pickle.dump(id2label, f)

with open(ENCODING_DIR / "label2id.pkl", "wb") as f:
    pickle.dump(label2id, f)

labels

{'SDG': ClassLabel(names=['1', '10', '11', '12', '13', '14', '15', '16', '17', '2', '3', '4', '5', '6', '7', '8', '9'], id=None), 'ABSTRACT': Value(dtype='string', id=None), 'id': Value(dtype='string', id=None), 'sdg_desc_short': Value(dtype='string', id=None), 'sdg_desc_long': Value(dtype='string', id=None)}
Example instance:	 {'SDG': 16, 'ABSTRACT': 'The first attempts to modernize simply replaced the single huge engine with a huge electric motor, changing little. The drive-shafts were replaced by wires, the huge steam engine by dozens of small motors. Factories spread out, there was natural light, and room to use ceiling-slung cranes. Workers had responsibility for their own machines, they needed better training and better pay. The electric motor was a wonderful invention, once we changed all the everyday details that surrounded it.', 'id': None, 'sdg_desc_short': None, 'sdg_desc_long': None}


{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16}

In [7]:
from transformers import AutoTokenizer

# whether the text should be lowered or not
SHOULD_LOWER = False

# base model
HF_MODEL_NAME = "meta-llama/Meta-Llama-3-8B"
# final model
MODEL_NAME = f"{HF_MODEL_NAME}-ft-{DATASET_TYPE.value}" + ("-lower" if SHOULD_LOWER else "") + "-unmasked"

tokenizer = AutoTokenizer.from_pretrained(HF_MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
# print(tokenizer.pad_token, tokenizer.eos_token)


def preprocess_data(instances):
    match DATASET_TYPE:
        case DatasetType.SWISSTEXT_SHARED_TASK1:
            # take a batch of titles and abstracts and concat them
            titles = instances["TITLE"]
            abstracts = instances["ABSTRACT"]
            texts = [f"{title} {abstract}" for title, abstract in zip(titles, abstracts)]
        case DatasetType.ZO_UP | DatasetType.TOY:
            texts = instances["ABSTRACT"]


    if SHOULD_LOWER:
        texts = [text.lower() for text in texts]

    # encode
    encoding = tokenizer(texts, padding="longest", truncation=True, max_length=1024, return_tensors="pt")

    # add labels
    encoding["label"] = torch.tensor([label for label in instances["SDG"]])

    return encoding

encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset["train"].column_names)
encoded_dataset.set_format("torch")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/dill/_dill.py:414: PicklingWarning: Cannot locate reference to <enum 'DatasetType'>.
  StockPickler.save(self, obj, save_persistent_id)
/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/dill/_dill.py:414: PicklingWarning: Cannot pickle <enum 'DatasetType'>: __main__.DatasetType has recursive self-references that trigger a RecursionError.
  StockPickler.save(self, obj, save_persistent_id)


## Model

In [25]:
from transformers.data import DataCollatorWithPadding
from peft import LoraConfig, TaskType, get_peft_model
from transformers import BitsAndBytesConfig


bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    #bnb_4bit_use_double_quant=True,
    #bnb_4bit_quant_type="nf4",
    #bnb_4bit_compute_dtype=torch.bfloat16
)

model = UnmaskingLlamaForSequenceClassification.from_pretrained(
    HF_MODEL_NAME,
    device_map="auto",
    quantization_config=bnb_config,
    num_labels=len(labels),
    token=HF_TOKEN
).bfloat16()
model.config.pad_token_id = tokenizer.pad_token_id
# model.resize_token_embeddings(len(tokenizer))

peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=16,
    lora_alpha=32,
    lora_dropout=0.1
)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()


data_collator = DataCollatorWithPadding(tokenizer)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of UnmaskingLlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Meta-Llama-3-8B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 6,885,376 || all params: 7,511,879,680 || trainable%: 0.0917


In [33]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import TrainingArguments, Trainer, EvalPrediction

BATCH_SIZE = 4
METRIC_NAME = "accuracy"

args = TrainingArguments(
    f"{CHECKPOINT_PATH}/{MODEL_NAME}",
    evaluation_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    save_steps=50,
    learning_rate=5e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=20,
    load_best_model_at_end=True,
    save_total_limit=2,
    metric_for_best_model=METRIC_NAME,
    seed=SEED,
    report_to="wandb",
    logging_steps=50,
)

def compute_metrics(pred: EvalPrediction):
    labels = pred.label_ids
    accuracy = accuracy_score(labels, pred.predictions.argmax(-1))
    precision, recall, f1, _ = precision_recall_fscore_support(labels, pred.predictions.argmax(-1), average="weighted")
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [38]:
from sklearn.metrics import classification_report

model.eval()

# manual evaluation to show classifcation_report
true_labels = []
logits = []

for batch in encoded_dataset["test"]:
    batch = {k: v.to(trainer.args.device).unsqueeze(0) for k, v in batch.items()}
    label = batch.pop("label")

    # Forward pass
    with torch.no_grad():
        out = model(**batch)

    true_labels.append(label.item())
    logits.extend(out.logits.tolist())

probabilites = torch.nn.functional.softmax(torch.tensor(logits), dim=-1)
pred_labels = torch.argmax(probabilites, dim=-1).tolist()

report = classification_report(true_labels, pred_labels, target_names=[f"SDG {id2label[i]}" for i in range(len(labels))])

In [39]:
print(report)

              precision    recall  f1-score   support

       SDG 1       0.71      0.71      0.71        17
      SDG 10       0.50      0.41      0.45        17
      SDG 11       0.82      0.82      0.82        17
      SDG 12       0.71      0.59      0.65        17
      SDG 13       0.83      0.88      0.86        17
      SDG 14       1.00      1.00      1.00        17
      SDG 15       0.80      0.94      0.86        17
      SDG 16       0.55      0.65      0.59        17
      SDG 17       0.00      0.00      0.00         1
       SDG 2       0.68      0.81      0.74        16
       SDG 3       0.80      0.94      0.86        17
       SDG 4       0.80      0.94      0.86        17
       SDG 5       0.74      1.00      0.85        17
       SDG 6       0.89      1.00      0.94        17
       SDG 7       1.00      0.65      0.79        17
       SDG 8       0.78      0.44      0.56        16
       SDG 9       0.83      0.59      0.69        17

    accuracy              

In [ ]:
# Loading the model for inference below

model = UnmaskingLlamaForSequenceClassification.from_pretrained(
    f"{CHECKPOINT_PATH}/{MODEL_NAME}/checkpoint-1850",
    num_labels=len(labels),
    device_map="auto",
    torch_dtype=torch.bfloat16
)
model.config.pad_token_id = tokenizer.pad_token_id